In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys as keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import time
import os
import shutil

In [2]:
class TwAutoGet:
  def __init__(self,ID,PASS,driver_path,max_elapsed=30):
    self.ID = ID
    self.PASS = PASS
    self.driver_path = driver_path
    self.max_elapsed = max_elapsed
    
  def initialize_driver(self):
    try:
      current_dir = os.getcwd()
      tmp_download_dir = f'{current_dir}/tmp_tweet_data'
      self.tmp_download_dir = tmp_download_dir
      
      if os.path.exists(tmp_download_dir):
        shutil.rmtree(tmp_download_dir)

      os.makedirs(tmp_download_dir, exist_ok=True)
      options = webdriver.ChromeOptions()
      prefs = {
        'download.default_directory' : tmp_download_dir,
        'profile.default_content_setting_values.automatic_downloads': 1
      }
      options.add_experimental_option('prefs',prefs)
      driver_path = '../Twitter/chromedriver'
      driver = webdriver.Chrome(executable_path = self.driver_path, options = options)
      self.driver = driver
    except Excepion as e:
      print("failed initialize")
      print(e)

  def login_twitter(self):
    URL="https://analytics.twitter.com/user/"+self.ID+"/tweets"
    username_input_path = '//input[@name="session[username_or_email]"]'
    password_input_path = '//input[@name="session[password]"]'
    driver = self.driver

    error_flg = False
    if error_flg is False:
      try:
        driver.get(URL)
        time.sleep(0.5)
        username_input = driver.find_element_by_xpath(username_input_path)
        username_input.send_keys(ID)
        time.sleep(0.5)
        password_input = driver.find_element_by_xpath(password_input_path)
        password_input.send_keys(PASS)
        time.sleep(0.5)
        username_input.submit()
        time.sleep(3)
        self.driver = driver

      except Exception as e:
        print('Login Error')
        print(e)
        error_flg = True

  def recursive_check(self, f_num_max, s_time):
    e_time = time.time()
    elapsed = e_time - s_time
    if elapsed > self.max_elapsed:
      return print("時間切れ")
    
    tmp_download_dir = self.tmp_download_dir
    f_num = sum(os.path.isfile(os.path.join(tmp_download_dir, name)) for name in os.listdir(tmp_download_dir))
    print("f_num: ", f_num)
    if f_num > f_num_max:
      print("先にファイルを削除してください")
    elif f_num == f_num_max:
      print("ダウンロードしました")
    else:
      time.sleep(5)
      print("... loading ...")
      return self.recursive_check(f_num_max,s_time)

  def download_analytics_data(self):
    self.initialize_driver()
    self.login_twitter()
    driver = self.driver
    print("driver: ", driver)
    
    export_btn_path = "//*[@id='export']/button"
    bytweet_btn_path = "//*[@id='export']/ul/li[1]/button"
    f_num_max = 0

    for n in [4,5,6,7]:
      print("n = ",n)
      driver.find_element_by_class_name('daterange-selected').click()
      time.sleep(1)

      month_btn_path = "/html/body/div[4]/div[4]/ul/li[{}]".format(n)
      month=driver.find_element_by_xpath(month_btn_path)
      month.click()
      time.sleep(1)
      export_btn=driver.find_element_by_xpath(export_btn_path)
      export_btn.click()
      time.sleep(1)
      bytweet_btn=driver.find_element_by_xpath(bytweet_btn_path)
      bytweet_btn.click()
      print("clicked!!!")
      time.sleep(1)

      f_num_max += 1
      print("f_num_max: ", f_num_max)
      s_time = time.time()
      self.recursive_check(f_num_max, s_time)

    return driver.quit()


In [3]:
ID = "myID"
PASS = "myPassWord"
driver_path = "./chromedriver"

TwAutoGet(ID,PASS,driver_path).download_analytics_data()

driver:  <selenium.webdriver.chrome.webdriver.WebDriver (session="859f3292cd3a235d08147e70be39f920")>
n =  4
clicked!!!
f_num_max:  1
f_num:  0
... loading ...
f_num:  1
ダウンロードしました
n =  5
clicked!!!
f_num_max:  2
f_num:  1
... loading ...
f_num:  1
... loading ...
f_num:  1
... loading ...
f_num:  2
ダウンロードしました
n =  6
clicked!!!
f_num_max:  3
f_num:  2
... loading ...
f_num:  2
... loading ...
f_num:  3
ダウンロードしました
n =  7
clicked!!!
f_num_max:  4
f_num:  3
... loading ...
f_num:  3
... loading ...
f_num:  3
... loading ...
f_num:  4
ダウンロードしました


In [6]:
import pandas as pd
import glob

In [14]:
files = glob.glob('tmp_tweet_data/*')
tw = pd.DataFrame()
for f in files:
  print(f)
  tw = pd.concat([tw,pd.read_csv(f)])
  
tw.head()

tmp_tweet_data/tweet_activity_metrics_haruchannna_20210101_20210201_ja.csv
tmp_tweet_data/tweet_activity_metrics_haruchannna_20210301_20210401_ja.csv
tmp_tweet_data/tweet_activity_metrics_haruchannna_20210201_20210301_ja.csv
tmp_tweet_data/tweet_activity_metrics_haruchannna_20210401_20210501_ja.csv


ツイートID                                         ツイートの固定リンク  \
0  1355792156825251842  https://twitter.com/haruchannna/status/1355792...   
1  1355791758341275654  https://twitter.com/haruchannna/status/1355791...   
2  1355547053611335683  https://twitter.com/haruchannna/status/1355547...   
3  1355546831938154500  https://twitter.com/haruchannna/status/1355546...   
4  1355546589654212612  https://twitter.com/haruchannna/status/1355546...   

                                              ツイート本文                      時間  \
0       @VtNpRvPfB7peEky みるくも毎日拭いてるんですけど全然治らないんですよね😂  2021-01-31 08:16 +0000   
1  @koubu33 鶏肉が涙やけの原因になることもあるんですね。参考になりました！ありがとうご...  2021-01-31 08:15 +0000   
2  @chip97857909 乾燥で涙焼けですか。調べてみたら充血してました。結膜炎なのかも。...  2021-01-30 16:02 +0000   
3  @washioemiko フード変えてみます。モグワンが涙やけに良いって聞いたので買ってみま...  2021-01-30 16:02 +0000   
4     @YFRPq0gxMehU1bp やっぱり病院行った方が良いですよね。ありがとうございます😊  2021-01-30 16:01 +0000   

   インプレッション  エンゲージメント  エンゲージメント率  リツイート   返信  いいね  ...  プロモのハッシュタグクリック  \
0      57.0       1.0   0.017544    0.0  1.0  0.0  ...               -   
1      34.0       0.0   0.000000    0.0  0.0  0.0  ...               -   
2     123.0       2.0   0.016260    0.0  1.0  1.0  ...               -   
3     176.0       5.0   0.028409    0.0  1.0  0.0  ...               -   
4      98.0       0.0   0.000000    0.0  0.0  0.0  ...               -   

   プロモの詳細クリック  プロモの固定リンクのクリック数  プロモのアプリ表示  プロモのアプリインストール  プロモのフォローしている  \
0           -                -          -              -             -   
1           -                -          -              -             -   
2           -                -          -              -             -   
3           -                -          -              -             -   
4           -                -          -              -             -   

   プロモのツイートをメール送信  プロモのダイアル式電話  プロモのメディアの再生数  プロモのメディアのエンゲージメント数  
0               -            -             -                   -  
1               -            -             -                   -  
2               -            -             -                   -  
3               -            -             -                   -  
4               -            -             -                   -  

[5 rows x 40 columns]